In [1]:
import time
import pygame
import pygame.midi
import math
import numpy
from json import load

import temsorflow as tf
from tensorflow import keras

pygame 2.1.2 (SDL 2.0.18, Python 3.9.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def print_midi_info():
    pygame.midi.init()
    for i in range(pygame.midi.get_count()):
        print(pygame.midi.get_device_info(i))
    
    print("Default in: ", pygame.midi.get_default_input_id())
    print("Default out: ", pygame.midi.get_default_output_id())
    pygame.midi.quit()

print_midi_info()

(b'MMSystem', b'Microsoft MIDI Mapper', 0, 1, 0)
(b'MMSystem', b'AKM322', 1, 0, 0)
(b'MMSystem', b'CoolSoft MIDIMapper', 0, 1, 0)
(b'MMSystem', b'Microsoft GS Wavetable Synth', 0, 1, 0)
(b'MMSystem', b'VirtualMIDISynth #1', 0, 1, 0)
(b'MMSystem', b'AKM322', 0, 1, 0)
Default in:  1
Default out:  0


In [3]:
notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def num_to_note(num):
    note = notes[num % 12]
    note = note + str(math.ceil(num/12)-1)
    return note

def read_model():
    existent_model = keras.load_model('models/best_model_note.h5')
    existent_model.summary()

In [4]:
def take_input(in_id = None, out_id = None):
    pygame.midi.init()

    if not in_id:
        in_id = pygame.midi.get_default_input_id()

    if not out_id:
        out_id = pygame.midi.get_default_output_id()

    i = pygame.midi.Input(in_id)
    o = pygame.midi.Output(out_id,latency=0,buffer_size=1)
    o.set_instrument(0)

    pygame.display.set_mode((1,1))
    note_count = 0

    #Turn these arrays into nupy.array()s like this: final_list = np.array(list(zip(note_seq,dur_seq)))
                # Will need to map ints from keyboard to chars representing the note (ie 'C3' 'A-4' etc) 
                # then return it and in produce_note_model create a new function and map the chars to their 
                # respective ints before sending them to produce_song 

                # have to caulculate the differnece in the start and end of notes, round the result to the 
                # nearest musical "pause" then send the word along with the string for note (bpm 120)
            #Chords?
                # PLeASe nO
        #I'll need to store the notes that have been pressed to check timestamp for release of the note


        #A mid C is 60, so using mod 12 to get notes and octave is important.

    #Format: [[pitch, start, stop], [pitch,start,stop], ...]
    note_seq = [] 

    while note_count < 10:
        if i.poll(): #if a note has been presseed
            midi_event = i.read(1) #grab note played

            if midi_event[0][0][0] == 144: #if note on
                print(midi_event)
                print(num_to_note(midi_event[0][0][1]))
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1] # in milliseconds I think

                note_seq.append([note, time_stamp, 0])

                o.note_on(note, 120)               

            if midi_event[0][0][0] == 128: #if note off
                print(midi_event)
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1]
                for n in range(len(note_seq)-1, -1, -1):
                    if(note_seq[n][0] == note and note_seq[n][2] == 0):
                        note_seq[n][2] = time_stamp
                o.note_off(note, 120)
               
                note_count += 1  #DOES NOT ACCOUNT FOR CHORDS

            #o.write_short(midi_event[0][0][0], midi_event[0][0][1], 120) #write start/stop, note, velocity to output
    print(note_seq)
    time.sleep(1) #Give last note time to play
    i.close()
    o.close()
    pygame.midi.quit()
    pygame.quit()

take_input(out_id=4)

read_model()
int_to_note = load("int_to_note.json")

[[[144, 55, 127, 0], 1339]]
G4
[[[128, 55, 0, 0], 1431]]
[[[144, 57, 92, 0], 1604]]
A4
[[[128, 57, 0, 0], 1674]]
[[[144, 55, 102, 0], 1871]]
G4
[[[128, 55, 0, 0], 1949]]
[[[144, 57, 92, 0], 2172]]
A4
[[[128, 57, 0, 0], 2236]]
[[[144, 55, 92, 0], 2455]]
G4
[[[128, 55, 0, 0], 2507]]
[[[144, 57, 90, 0], 2715]]
A4
[[[128, 57, 0, 0], 2774]]
[[[144, 55, 92, 0], 2997]]
G4
[[[128, 55, 0, 0], 3064]]
[[[144, 57, 90, 0], 3276]]
A4
[[[128, 57, 0, 0], 3326]]
[[[144, 55, 90, 0], 3537]]
G4
[[[128, 55, 0, 0], 3600]]
[[[144, 57, 90, 0], 3820]]
A4
[[[128, 57, 0, 0], 3869]]
[[55, 1339, 0], [57, 1604, 1674], [55, 1871, 1949], [57, 2172, 2236], [55, 2455, 2507], [57, 2715, 2774], [55, 2997, 3064], [57, 3276, 3326], [55, 3537, 3600], [57, 3820, 3869]]
